In [3]:
!pip install langchain
!pip install pandas

In [5]:
import os
import logging
from langchain.text_splitter import MarkdownTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import NotebookLoader
from langchain.document_loaders import CSVLoader
import json
from dotenv import load_dotenv

load_dotenv()
logging.basicConfig(level=logging.INFO)

In [26]:
# 파일 경로 설정
notebook_path = './extract_vec_ipynb.ipynb'
csv_path = './subwaystation.csv'

# .csv 파일을 불러오기
import pandas as pd
from langchain.schema import Document
from langchain.text_splitter import MarkdownTextSplitter

# CSV 파일 읽기
df = pd.read_csv(csv_path, encoding='utf-8', encoding_errors='replace')  # 인코딩 문제를 피하기 위해 'errors' 인자를 사용

# CSV 파일의 각 행을 Document 형식으로 변환
documents = [
    Document(page_content=row.to_string(), metadata={'source_file': 'subwaystation.csv', 'source_folder': 'mini'})
    for _, row in df.iterrows()
]

# 텍스트 분할 설정
text_splitter = MarkdownTextSplitter(chunk_size=600, chunk_overlap=200)

# 분할된 문서 리스트 생성
docs = []
for doc in documents:
    split_docs = text_splitter.create_documents([doc.page_content])
    for split_doc in split_docs:
        split_doc.metadata = doc.metadata
    docs.extend(split_docs)

# 분할된 문서 예시 출력
for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]\n")
    print("Content:")
    print(doc.page_content)
    print("\nMetadata:")
    print(doc.metadata) 
    print("=" * 60)


[Chunk 0]

Content:
역번호                                          0124
역사명                                  청량리(서울시립대입구)
노선명                                           1호선
환승역구분                                    도시철도 환승역
환승노선명    수도권  광역철도 분당+수도권  광역철도 경춘+수도권  광역철도 경의중앙

Metadata:
{'source_file': 'subwaystation.csv', 'source_folder': 'mini'}
[Chunk 1]

Content:
역번호          0125
역사명           제기동
노선명           1호선
환승역구분    도시철도 일반역
환승노선명         NaN

Metadata:
{'source_file': 'subwaystation.csv', 'source_folder': 'mini'}
[Chunk 2]

Content:
역번호               0126
역사명                신설동
노선명                1호선
환승역구분         도시철도 환승역
환승노선명    수도권  도시철도 2호선

Metadata:
{'source_file': 'subwaystation.csv', 'source_folder': 'mini'}
[Chunk 3]

Content:
역번호               0127
역사명                동묘앞
노선명                1호선
환승역구분         도시철도 환승역
환승노선명    수도권  도시철도 6호선

Metadata:
{'source_file': 'subwaystation.csv', 'source_folder': 'mini'}
[Chunk 4]

Content:
역번호               0128
역사명            

In [27]:
# Step 4: 임베딩 함수 초기화 및 FAISS 벡터 스토어 저장
try:
    logging.info("Initializing embedding function")
    embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")

    logging.info("Adding documents to FAISS vector store")
    vectorstore = FAISS.from_documents(
        documents=docs,
        embedding=embedding_function
    )
    logging.info("Documents added to FAISS vector store")

    # 로컬 디렉터리에 FAISS 벡터 스토어 저장
    vectorstore.save_local('./langagent/faiss')
    logging.info("FAISS vector store saved locally to './langagent/faiss'")

    # 메타데이터 저장
    metadata = [doc.metadata for doc in docs]  # 모든 문서의 메타데이터 수집
    with open('./langagent/faiss/metadata.json', 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=4)
    logging.info("Metadata saved locally to './langagent/faiss/metadata.json'")

except Exception as e:
    logging.error(f"Error adding documents to FAISS vector store: {e}")
    exit(1)

INFO:root:Initializing embedding function
INFO:root:Adding documents to FAISS vector store
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:root:Documents added to FAISS vector store
INFO:root:FAISS vector store saved locally to './langagent/faiss'
INFO:root:Metadata saved locally to './langagent/faiss/metadata.json'


In [43]:

import logging
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

from dotenv import load_dotenv
load_dotenv()

embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.load_local('./langagent/faiss', embedding_function, allow_dangerous_deserialization=True)
logging.info("FAISS vector store loaded from './langagent/faiss'")

def query_restaurant_recommendations(user_input):
    try:
        logging.info(f"Starting query search: {user_input}")

        # Use the vectorstore's as_retriever method
        retriever = vectorstore.as_retriever(search_type="similarity")
        search_results = retriever.get_relevant_documents(user_input)

        logging.info("Query search completed")

        cleaned_results = []
        for doc in search_results:
            content = doc.page_content.strip()
            metadata = doc.metadata 
            cleaned_results.append({
                "content": content,
                "metadata": metadata 
            })
        return cleaned_results

    except Exception as e:
        logging.error(f"Error during query search: {e}")
        return []

INFO:root:FAISS vector store loaded from './langagent/faiss'


In [45]:
query_restaurant_recommendations('혜화역에서 가산디지털단지역까지 어떻게 가?')

INFO:root:Starting query search: 혜화역에서 가산디지털단지역까지 어떻게 가?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:root:Query search completed


[{'content': '역번호               0746\n역사명            가산디지털단지\n노선명                7호선\n환승역구분         도시철도 환승역\n환승노선명    수도권  광역철도 1호선',
  'metadata': {'source_file': 'subwaystation.csv', 'source_folder': 'mini'}},
 {'content': '역번호          1702\n역사명      가산디지털단지역\n노선명           경부선\n환승역구분         환승역\n환승노선명         7호선',
  'metadata': {'source_file': 'subwaystation.csv', 'source_folder': 'mini'}},
 {'content': '역번호           1266\n역사명      디지털미디어시티역\n노선명          경의중앙선\n환승역구분          환승역\n환승노선명      6호선, 공항',
  'metadata': {'source_file': 'subwaystation.csv', 'source_folder': 'mini'}},
 {'content': '역번호                    A04\n역사명               디지털미디어시티\n노선명                인천국제공항선\n환승역구분                  환승역\n환승노선명    서울 도시철도 6호선+경의중앙선',
  'metadata': {'source_file': 'subwaystation.csv', 'source_folder': 'mini'}}]